In [3]:
import torch
from transformers import AutoTokenizer, AutoModel

DEVICE = torch.device('cuda')

tokenizer = AutoTokenizer.from_pretrained("cointegrated/rubert-tiny2", cache_dir='rubert')
model = AutoModel.from_pretrained("cointegrated/rubert-tiny2", cache_dir='rubert').to(DEVICE)
# model.cuda()  # uncomment it if you have a GPU

Some weights of the model checkpoint at cointegrated/rubert-tiny2 were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


(312,)


In [11]:
def embed_bert_cls(text, model, tokenizer):
    t = tokenizer(text, padding=True, truncation=True, return_tensors='pt')
    with torch.no_grad():
        model_output = model(**{k: v.to(model.device) for k, v in t.items()})
    embeddings = model_output.last_hidden_state[:, 0, :]
    embeddings = torch.nn.functional.normalize(embeddings)
    return embeddings[0].cpu().numpy()

true embeds

In [171]:
import pandas as pd

df = pd.read_csv('lenta_tech.csv')

In [172]:
from tqdm.notebook import tqdm

In [173]:
embeds = []
for text in tqdm(df.text.values):
    embeds.append(embed_bert_cls(text, model, tokenizer))

  0%|          | 0/18832 [00:00<?, ?it/s]

In [174]:
import numpy as np
embeds = np.array(embeds)

In [176]:
np.save('title_embeds.npy', embeds)

In [ ]:
print(embed_bert_cls('привет мир', model, tokenizer).shape)

gen embeds

In [56]:
generated = pd.read_csv('generated.txt', header=None, squeeze=True)

In [58]:
gen_embeds = []
for text in tqdm(generated.values):
    gen_embeds.append(embed_bert_cls(text, model, tokenizer))

  0%|          | 0/2000 [00:00<?, ?it/s]

In [59]:
import numpy as np
gen_embeds = np.array(gen_embeds)

In [60]:
np.save('gen_embeds.npy', gen_embeds)

comparing